In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uit-diabetic-patients-re-admission-prediction/IDs_mapping.csv
/kaggle/input/uit-diabetic-patients-re-admission-prediction/test_diabetic_data.csv
/kaggle/input/uit-diabetic-patients-re-admission-prediction/train_diabetic_data.csv
/kaggle/input/uit-diabetic-patients-re-admission-prediction/2014 beata strack et al.pdf
/kaggle/input/uit-diabetic-patients-re-admission-prediction/2016 bhuvan m s et al.pdf
/kaggle/input/uit-diabetic-patients-re-admission-prediction/sample_solution.csv


In [2]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=d254bf4f55c203931117e16c6203a5c8adc4f98fdf3eece1bb36240f80eb0ed4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import findspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/20 10:56:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
import scipy.stats as stats
from pyspark.ml.stat import ChiSquareTest
import pyspark
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, StringIndexerModel, IndexToString, OneHotEncoder

# **LOAD DATASET**

In [5]:
train = spark.read.format("csv").load("/kaggle/input/uit-diabetic-patients-re-admission-prediction/train_diabetic_data.csv",header = 'True',inferSchema='True')
test = spark.read.format("csv").load("/kaggle/input/uit-diabetic-patients-re-admission-prediction/test_diabetic_data.csv",header = 'True',inferSchema='True')

In [6]:
drugs = ['examide','metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
         'glimepiride', 'acetohexamide', 'glipizide','glyburide', 'tolbutamide',
         'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
        'tolazamide', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
        'metformin-rosiglitazone', 'metformin-pioglitazone', 'citoglipton']

# **Data Preprocessing**

In [7]:
def data_preprocess(df):
    # Replace '?' with None
    df = df.replace('?', None)

    # Handle missing values in the 'race' column
    df = df.withColumn('race', when(F.col('race').isNull(), 'Other').otherwise(F.col('race')))
    
    #1: eme, 5: not, 3:ele
    df = df.withColumn(
        'admission_type_id',
        when(F.col('admission_type_id').isin([2, 7]), 1)
        .when(F.col('admission_type_id').isin([6, 8]), 5)
        .otherwise(F.col('admission_type_id'))
    )

    # Replace values in 'discharge_disposition_id'
    df = df.withColumn(
        'discharge_disposition_id',
        when(F.col('discharge_disposition_id').isin([6, 8, 9, 13]), 1)
        .when(F.col('discharge_disposition_id').isin([3, 4, 5, 14, 22, 23, 24]), 2)
        .when(F.col('discharge_disposition_id').isin([12, 15, 16, 17]), 10)
        .when(F.col('discharge_disposition_id').isin([25, 26]), 18)
        .otherwise(F.col('discharge_disposition_id'))
    )

    # Replace values in 'admission_source_id' 7:eme, 9:not
    df = df.withColumn(
        'admission_source_id',
        when(F.col('admission_source_id').isin([2, 3]), 1)
        .when(F.col('admission_source_id').isin([5, 6, 10, 22, 25]), 4)
        .when(F.col('admission_source_id').isin([15, 17, 20, 21]), 9)
        .when(F.col('admission_source_id').isin([13, 14]), 11)
        .otherwise(F.col('admission_source_id'))
    )
    
    df = df.withColumn(
    'admission_source_id',
    when((col('admission_source_id') == 9) & (col('admission_type_id') == 1), 7)
    .otherwise(col('admission_source_id'))
    )
    
    df = df.withColumn(
    'admission_source_id',
    when((col('admission_source_id') == 9) & (col('admission_type_id') == 3), 1)
    .otherwise(col('admission_source_id'))
    )
    
    df = df.withColumn('number_diab_meds', F.lit(0).cast('int'))

    for j in drugs:
        condition = (col(j) != 'No')
        df = df.withColumn('number_diab_meds', F.when(condition, col('number_diab_meds') + 1).otherwise(col('number_diab_meds')))

    df = df.withColumn('number_diab_meds', col('number_diab_meds').cast('int'))
    df = df.withColumn(
    'insulin_treatment',
    when((col('insulin') != 'No') & (col('number_diab_meds') == 1), 'insulin_only')
    .when((col('insulin') != 'No') & (col('number_diab_meds') > 1), 'insulin_combo')
    .when((col('insulin') == 'No') & (col('number_diab_meds') == 0), 'no_med')
    .otherwise('other_meds')
    )
    
    df = df.withColumn(
    'preceding_year_visits',
    col('number_outpatient') + col('number_emergency') + col('number_inpatient')
    )
    
    for i in drugs:
        df = df.withColumn(
            i,
            when(col(i) == 'No', -2)
            .when(col(i) == 'Down', -1)
            .when(col(i) == 'Steady', 0)
            .when(col(i) == 'Up', 1)
            .otherwise(col(i))
        )
    
    df = df.drop('weight','payer_code', 'medical_specialty', 'number_outpatient', 'number_emergency', 'number_inpatient', 'admission_type_id')
    
    for i in drugs:
        df = df.drop(i)
    return df

In [8]:
def grouper_udf(i):
    if i is None:
        return "unknown"

    i = str(i)

    if i[0].isnumeric():
        i = float(i)

        if 1 <= i <= 139:
            return "001-139"
        elif 140 <= i <= 239:
            return "140-239"
        elif 240 <= i <= 279:
            return "240-279"
        elif 280 <= i <= 289:
            return "280-289"
        elif 290 <= i <= 319:
            return "290-319"
        elif 320 <= i <= 389:
            return "320-389"
        elif 390 <= i <= 459:
            return "390-459"
        elif 460 <= i <= 519:
            return "460-519"
        elif 520 <= i <= 579:
            return "520-579"
        elif 580 <= i <= 629:
            return "580-629"
        elif 630 <= i <= 679:
            return "630-679"
        elif 680 <= i <= 709:
            return "680-709"
        elif 710 <= i <= 739:
            return "710-739"
        elif 740 <= i <= 759:
            return "740-759"
        elif 760 <= i <= 779:
            return "760-779"
        elif 780 <= i <= 799:
            return "780-799"
        else:
            return "800-999"
    else:  # if the code does not begin with a number
        return "EV_code"

# Register the UDF
grouper = udf(grouper_udf, StringType())

In [9]:
def count_nulls(c):
    """Input pyspark dataframe and return list of columns with missing value and it's total value"""
    null_counts = []
    for col in c.dtypes:
        cname = col[0]
        ctype = col[1]
        nulls = c.where( c[cname].isNull()).count()
        result = tuple([cname, nulls])
        null_counts.append(result)
    null_counts=[(x,y) for (x,y) in null_counts if y!=0]
    return null_counts

# Train Data

In [10]:
train = data_preprocess(train)
# Apply the UDF to create new grouped columns
train = train.withColumn("diag_1_grouped", grouper(train["diag_1"]))
train = train.withColumn("diag_2_grouped", grouper(train["diag_2"]))
train = train.withColumn("diag_3_grouped", grouper(train["diag_3"]))
train = train.drop('diag_1', 'diag_2', 'diag_3')

In [11]:
train.groupBy('readmitted').count().show()

+----------+-----+
|readmitted|count|
+----------+-----+
|       >30|13920|
|        NO|54374|
|       <30| 3224|
+----------+-----+



In [12]:
train.groupBy('readmitted').count().show()

+----------+-----+
|readmitted|count|
+----------+-----+
|       >30|13920|
|        NO|54374|
|       <30| 3224|
+----------+-----+



In [13]:
grouped_data = train.groupBy("readmitted").count()

class_0 = train.filter(col("readmitted") == "NO")
class_1 = train.filter(col("readmitted") == ">30")
class_2 = train.filter(col("readmitted") == "<30")
class_0_resampled = class_0.sample(True, 1000 / class_0.count(), seed=42)
class_1_resampled = class_1.sample(True, 3224 / class_1.count(), seed=42)
balanced_df = class_0_resampled.union(class_1).union(class_2)

In [14]:
balanced_df.groupBy('readmitted').count().show()

+----------+-----+
|readmitted|count|
+----------+-----+
|        NO| 1041|
|       >30|13920|
|       <30| 3224|
+----------+-----+



In [15]:
train = balanced_df

In [16]:
train.printSchema()

root
 |-- encounter_id: integer (nullable = true)
 |-- patient_nbr: integer (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- discharge_disposition_id: integer (nullable = true)
 |-- admission_source_id: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: string (nullable = true)
 |-- A1Cresult: string (nullable = true)
 |-- insulin: string (nullable = true)
 |-- change: string (nullable = true)
 |-- diabetesMed: string (nullable = true)
 |-- readmitted: string (nullable = true)
 |-- number_diab_meds: integer (nullable = false)
 |-- insulin_treatment: string (nullable = false)
 |-- preceding_year_visits: integer (nullable = true)
 |-- diag_1_grouped: string (nullable = tr

# Test Data

In [17]:
test = data_preprocess(test)
# Apply the UDF to create new grouped columns
test = test.withColumn("diag_1_grouped", grouper(test["diag_1"]))
test = test.withColumn("diag_2_grouped", grouper(test["diag_2"]))
test = test.withColumn("diag_3_grouped", grouper(test["diag_3"]))
test = test.drop('diag_1', 'diag_2', 'diag_3')

In [18]:
test.printSchema()

root
 |-- index: integer (nullable = true)
 |-- encounter_id: integer (nullable = true)
 |-- patient_nbr: integer (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- discharge_disposition_id: integer (nullable = true)
 |-- admission_source_id: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: string (nullable = true)
 |-- A1Cresult: string (nullable = true)
 |-- insulin: string (nullable = true)
 |-- change: string (nullable = true)
 |-- diabetesMed: string (nullable = true)
 |-- number_diab_meds: integer (nullable = false)
 |-- insulin_treatment: string (nullable = false)
 |-- preceding_year_visits: integer (nullable = true)
 |-- diag_1_grouped: string (nullable = true)


# **Data Indexer**

In [19]:
def get_string_columns(df):
    string_columns = [col.name for col in df.schema if col.dataType == StringType() and col.name != 'readmitted']
    return string_columns

In [20]:
cat_col = get_string_columns(train)
cat_col

['race',
 'gender',
 'age',
 'max_glu_serum',
 'A1Cresult',
 'insulin',
 'change',
 'diabetesMed',
 'insulin_treatment',
 'diag_1_grouped',
 'diag_2_grouped',
 'diag_3_grouped']

In [21]:
from pyspark.ml.feature import Imputer
indexer_stages = []

labelIndexer = StringIndexer(inputCol='readmitted', outputCol="label")
indexedLabelTrain=labelIndexer.fit(train).transform(train)
indexedLabelTrain = indexedLabelTrain.drop('readmitted')

cat_col = get_string_columns(indexedLabelTrain)
for col_name in cat_col:
    indexer = StringIndexer(inputCol=col_name, outputCol=f"indexed_{col_name}", handleInvalid="keep")
    indexer_stages.append(indexer)

indexed_df = indexedLabelTrain
for index in indexer_stages:
  indexed_df = index.fit(indexed_df).transform(indexed_df)
df_transformed = indexed_df.drop(*cat_col)

In [22]:
indexedLabelTrain.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  2.0| 1041|
|  0.0|13920|
|  1.0| 3224|
+-----+-----+



In [23]:
df_transformed.printSchema()

root
 |-- encounter_id: integer (nullable = true)
 |-- patient_nbr: integer (nullable = true)
 |-- discharge_disposition_id: integer (nullable = true)
 |-- admission_source_id: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- number_diab_meds: integer (nullable = false)
 |-- preceding_year_visits: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- indexed_race: double (nullable = false)
 |-- indexed_gender: double (nullable = false)
 |-- indexed_age: double (nullable = false)
 |-- indexed_max_glu_serum: double (nullable = false)
 |-- indexed_A1Cresult: double (nullable = false)
 |-- indexed_insulin: double (nullable = false)
 |-- indexed_change: double (nullable = false)
 |-- indexed_diabetesMed: double (nullable = false)
 |-- indexed_insulin_tre

In [24]:
feature_columns = [col for col in df_transformed.columns if col != "label"]
feature_columns

['encounter_id',
 'patient_nbr',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_diagnoses',
 'number_diab_meds',
 'preceding_year_visits',
 'indexed_race',
 'indexed_gender',
 'indexed_age',
 'indexed_max_glu_serum',
 'indexed_A1Cresult',
 'indexed_insulin',
 'indexed_change',
 'indexed_diabetesMed',
 'indexed_insulin_treatment',
 'indexed_diag_1_grouped',
 'indexed_diag_2_grouped',
 'indexed_diag_3_grouped']

In [25]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df2 = assembler.transform(df_transformed)
df2.show(2)

23/12/20 10:57:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+-----------+------------------------+-------------------+----------------+------------------+--------------+---------------+----------------+----------------+---------------------+-----+------------+--------------+-----------+---------------------+-----------------+---------------+--------------+-------------------+-------------------------+----------------------+----------------------+----------------------+--------------------+
|encounter_id|patient_nbr|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_diagnoses|number_diab_meds|preceding_year_visits|label|indexed_race|indexed_gender|indexed_age|indexed_max_glu_serum|indexed_A1Cresult|indexed_insulin|indexed_change|indexed_diabetesMed|indexed_insulin_treatment|indexed_diag_1_grouped|indexed_diag_2_grouped|indexed_diag_3_grouped|            features|
+------------+-----------+------------------------+-------------------+----------------+----------------

In [26]:
from pyspark.ml.classification import DecisionTreeClassifier,NaiveBayes, RandomForestClassifier, MultilayerPerceptronClassifier
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
model = RandomForestClassifier(labelCol="label", featuresCol="scaled_features", maxDepth=9, numTrees=150)
all_stages = indexer_stages + [assembler, scaler, model]
pipeline = Pipeline(stages=all_stages)


# train_data, test_data = indexedLabelTrain.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(indexedLabelTrain)

23/12/20 10:58:36 WARN DAGScheduler: Broadcasting large task binary with size 1233.6 KiB
23/12/20 10:58:38 WARN DAGScheduler: Broadcasting large task binary with size 1918.0 KiB
23/12/20 10:58:41 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
23/12/20 10:58:45 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
23/12/20 10:58:49 WARN DAGScheduler: Broadcasting large task binary with size 1127.5 KiB
23/12/20 10:58:51 WARN DAGScheduler: Broadcasting large task binary with size 8.3 MiB
23/12/20 10:58:56 WARN DAGScheduler: Broadcasting large task binary with size 1689.2 KiB


In [27]:
# train_data.groupBy('label').count().show()

In [28]:
# test_data.groupBy('label').count().show()

In [29]:
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# predictions = model.transform(test_data)

# accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
# accuracy = accuracy_evaluator.evaluate(predictions)
# print(f"Accuracy: {accuracy}")

# # Evaluate F1 score
# f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
# f1_score = f1_evaluator.evaluate(predictions)
# print(f"F1 Score: {f1_score}")

In [30]:
testSolutions = model.transform(test).select('index','prediction')
testSolutions.show()

labelsArray = ['GT30' , 'LT30' , 'NO']
testSolutions = IndexToString(inputCol="prediction", outputCol="readmitted", labels = labelsArray).transform(testSolutions)

23/12/20 10:59:05 WARN DAGScheduler: Broadcasting large task binary with size 5.9 MiB


+-----+----------+
|index|prediction|
+-----+----------+
|   12|       0.0|
|   27|       0.0|
|   28|       0.0|
|   32|       0.0|
|   33|       0.0|
|   35|       0.0|
|   37|       0.0|
|   43|       0.0|
|   46|       0.0|
|   50|       0.0|
|   60|       0.0|
|   68|       0.0|
|   71|       0.0|
|   75|       0.0|
|   79|       0.0|
|   82|       0.0|
|   91|       0.0|
|   94|       0.0|
|   96|       0.0|
|  110|       0.0|
+-----+----------+
only showing top 20 rows



In [31]:
solutions = testSolutions.select('index','readmitted')
solutions.toPandas().to_csv("dumpsolutions.csv",header=True, index=False)

23/12/20 10:59:07 WARN DAGScheduler: Broadcasting large task binary with size 5.9 MiB


In [32]:
solutions.groupBy('readmitted').count().show()

23/12/20 10:59:14 WARN DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/12/20 10:59:19 WARN DAGScheduler: Broadcasting large task binary with size 5.9 MiB


+----------+-----+
|readmitted|count|
+----------+-----+
|      LT30|   33|
|        NO|    4|
|      GT30|30211|
+----------+-----+

